# Logic Graph Examples

Create interactive graphs of logical formulas showing inference relationships.

## Two Approaches

**Syntactic Derivability** (`build_syntactic_graph`): Proof-based, forward-chaining
- Starts from axioms and applies inference rules step-by-step
- Shows HOW formulas are proven (derivation paths)
- Focused and efficient (20-100 formulas typical)
- Best for: Teaching logic, understanding proofs, working with many variables

**Semantic Entailment** (`build_semantic_graph`): Truth-based, exhaustive enumeration
- Generates all formulas, checks which are true under axioms
- Finds ALL logical consequences (complete for propositional logic)
- Comprehensive (1000s of formulas)
- Best for: Finding all consequences, exploring formula space, 2-3 variables

**Key insight:** Syntactic ⊆ Semantic — Everything provable is semantically entailed, but not vice versa.

In [1]:
# Example 1: Modus Ponens (Syntactic Derivability)
from calculus_ratiocinator import Var, Implies, build_syntactic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph using syntactic derivability (proof-based)
g = build_syntactic_graph(["x", "y"], axioms, max_iterations=10)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed (it should be via modus ponens)
if str(y) in g.nodes:
    print(f"✓ y is derived: {g.nodes[str(y)]['entailed']}")
    print(f"  Generation: {g.nodes[str(y)]['generation']}")

# Export to interactive HTML (automatically saved in graphs/ folder)
export_to_html(g, "modus_ponens_syntactic.html")

Graph has 12 nodes and 16 edges
✓ y is derived: True
  Generation: 1
Interactive graph saved to graphs\modus_ponens_syntactic.html


In [3]:
# Example 2: Chain of Implications (Syntactic Derivability)
from calculus_ratiocinator import Var, Implies, build_syntactic_graph, export_to_html

p = Var("p")
q = Var("q")
r = Var("r")

# Axioms: p, (p → q), (q → r)
axioms = [p, Implies(p, q), Implies(q, r)]

g = build_syntactic_graph(["p", "q", "r"], axioms, max_iterations=5)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

export_to_html(g, "chain_syntactic.html")

Graph has 30 nodes and 45 edges
Interactive graph saved to graphs\chain_syntactic.html


In [5]:
# Example 3: Conjunction Elimination (Syntactic Derivability)
from calculus_ratiocinator import Var, And, build_syntactic_graph, export_to_html

a = Var("a")
b = Var("b")

# Start with a conjunction axiom
axioms = [And(a, b)]

g = build_syntactic_graph(["a", "b"], axioms=axioms, max_iterations=10)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

export_to_html(g, "conjunction_syntactic.html")

Graph has 5 nodes and 4 edges
Interactive graph saved to graphs\conjunction_syntactic.html


In [3]:
# Example 4: Hilbert Axioms (Entailment)
from calculus_ratiocinator import (
    Var, Not, Implies,
    build_syntactic_graph,
    export_to_html,
)

def hilbert_axioms():
    p, q, r = Var("p"), Var("q"), Var("r")

    A1 = Implies(p, Implies(q, p))

    A2 = Implies(
        Implies(p, Implies(q, r)),
        Implies(Implies(p, q), Implies(p, r))
    )

    A3 = Implies(
        Implies(Not(q), Not(p)),
        Implies(p, q)
    )

    return [A1, A2, A3]

axioms = hilbert_axioms()
g = build_syntactic_graph(
    var_names=["p", "q", "r"],
    axioms=axioms,
    max_iterations=10,  # equivalent to proof search depth
)
print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")
export_to_html(g, "hilbert_syntactic.html")

Graph has 6 nodes and 5 edges
Interactive graph saved to graphs\hilbert_syntactic.html


# Semantic Entailment Approach

The examples above use **syntactic derivability** (proof-based): start from axioms and apply inference rules to derive new statements step-by-step.

Below we demonstrate **semantic entailment** (truth-based): generate all possible formulas up to a depth, then check which are true in all models where axioms hold.

**Key difference:** Semantic approach finds ALL logically entailed formulas (complete), while syntactic only derives what's provable with the implemented inference rules (incomplete but focused).

In [7]:
# Example 5: Modus Ponens (Semantic Entailment)
from calculus_ratiocinator import Var, Implies, build_semantic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph using semantic entailment (truth-based)
g = build_semantic_graph(["x", "y"], axioms, max_depth=2)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed
if str(y) in g.nodes:
    print(f"✓ y is semantically entailed: {g.nodes[str(y)]['entailed']}")

# Export to interactive HTML
export_to_html(g, "modus_ponens_semantic.html")

Graph has 786 nodes and 3697 edges
✓ y is semantically entailed: True
Interactive graph saved to graphs\modus_ponens_semantic.html


In [4]:
# Example 6: Hilbert Axioms (Semantic Entailment)
from calculus_ratiocinator import Var, Implies, build_semantic_graph, export_to_html

# Simple Hilbert axiom system
x = Var("x")
y = Var("y")
z = Var("z")

axioms = [
    Implies(x, Implies(y, x)),  # K axiom
    Implies(Implies(x, Implies(y, z)), Implies(Implies(x, y), Implies(x, z)))  # S axiom
]

# Build graph using semantic entailment
g = build_semantic_graph(["x", "y", "z"], axioms, max_depth=2)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Export to interactive HTML
export_to_html(g, "hilbert_semantic.html")

Graph has 3303 nodes and 1976 edges
Interactive graph saved to graphs\hilbert_semantic.html


## Understanding the Graph

**Node colors:**
- 🟠 Orange: Axiom formulas (starting points)
- 🟢 Light Green: Tautologies (always true)
- 🔵 Sky Blue: Entailed from axioms
- 🔴 Red: Not entailed (false under axioms)

**Edge colors (inference rules):**
- Dark Gray-Blue: General entailment
- Hot Pink: Modus Ponens (MP)
- Deep Purple: Modus Tollens (MT)
- Amber: Disjunctive Syllogism (DS)
- Cyan: Hypothetical Syllogism (HS)
- Indigo: Conjunction Elimination Left (∧E-L)
- Brown: Conjunction Elimination Right (∧E-R)

**Interactive features:**
- Drag nodes to rearrange
- Hover over nodes/edges for details
- Scroll to zoom
- Use navigation buttons in bottom-right

**Smart Sampling:**
- Perfect for exploring depth 3-4 without overwhelming the browser!
- Set `max_nodes` to limit graph size
- Prioritizes: axioms → entailed → tautologies → false statements (simpler first)